In [6]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse
from sklearn import metrics
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler

In [3]:
def query(parametro):
    query = f'''
        SELECT 
            {parametro} Periodo,
            sum(Cantidad) almuerzos

        FROM 
            (
                SELECT
                    Dia_del_Año,
                    Numero_de_Mes,
                    Numero_de_Semana,
                    Año,
                    id_cliente,
                    Boleta,
                    id_producto,
                    Cantidad,
                    Total,
                    Fecha
                FROM ingresos
                WHERE 1=1
                    AND Año = 2023
                    AND Numero_de_Mes >= 3
                    AND Tipo LIKE "ALMUERZOS"
            ) as sub
        GROUP BY 1
    '''
    return query

In [11]:
load_dotenv()

url = f"mysql+mysqlconnector://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
engine = create_engine(url=url)

df = pd.read_sql(query('Fecha'), con=engine)

df['Periodo'] = pd.to_datetime(df['Periodo'])
df = df.set_index(df['Periodo'])
df = df[['almuerzos']]

df.head()

,almuerzos
Periodo,
2023-03-01,8.0
2023-03-02,7.0
2023-03-03,5.0
2023-03-04,6.0
2023-03-05,12.0


In [13]:
scaler = MinMaxScaler()

train_data = df[(df.index<'2023-12-01')]
test_data = df[(df.index>='2023-12-01')]

train_data.shape, test_data.shape

((267, 1), (31, 1))

In [16]:
scaler.fit_transform(train_data)
scaled_train_data = scaler.transform(train_data)

scaled_test_data = scaler.transform(test_data)

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator